In [1]:
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import json
import pandas as pd
from IPython.display import display
from tqdm import tqdm
import sentencepiece as spm
import wget

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
vocab_file = "./ko_sentencepiece/ko.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

Config 설정

In [3]:
class Config(dict): 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)

In [4]:
config = Config({
    "n_enc_vocab": len(vocab),
    "n_dec_vocab": len(vocab),
    "n_enc_seq": 256,
    "n_dec_seq": 256,
    "n_layer": 6,
    "d_hidn": 256,
    "i_pad": 0,
    "d_ff": 1024,
    "n_head": 4,
    "d_head": 64,
    "dropout": 0.1,
    "layer_norm_epsilon": 1e-12
})
print(config)

{'n_enc_vocab': 8007, 'n_dec_vocab': 8007, 'n_enc_seq': 256, 'n_dec_seq': 256, 'n_layer': 6, 'd_hidn': 256, 'i_pad': 0, 'd_ff': 1024, 'n_head': 4, 'd_head': 64, 'dropout': 0.1, 'layer_norm_epsilon': 1e-12}


Transformer 구현

In [5]:
def get_sinusoid_encoding_table(n_seq, d_hidn):
    def cal_angle(position, i_hidn):
        return position / np.power(10000, 2 * (i_hidn // 2) / d_hidn)
    def get_posi_angle_vec(position):
        return [cal_angle(position, i_hidn) for i_hidn in range(d_hidn)]

    sinusoid_table = np.array([get_posi_angle_vec(i_seq) for i_seq in range(n_seq)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # even index sin 
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # odd index cos

    return sinusoid_table

def get_attn_pad_mask(seq_q, seq_k, i_pad):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(i_pad).unsqueeze(1).expand(batch_size, len_q, len_k)  # 
    return pad_attn_mask

def get_attn_decoder_mask(seq):
    subsequent_mask = torch.ones_like(seq).unsqueeze(-1).expand(seq.size(0), seq.size(1), seq.size(1))
    subsequent_mask = subsequent_mask.triu(diagonal=1) # upper triangular part of a matrix(2-D)
    return subsequent_mask

In [6]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.dropout = nn.Dropout(config.dropout)
        self.scale = 1 / (self.config.d_head ** 0.5)
    
    def forward(self, Q, K, V, attn_mask):
        # (bs, n_head, n_q_seq, n_k_seq)
        scores = torch.matmul(Q, K.transpose(-1, -2)).mul_(self.scale)
        scores.masked_fill_(attn_mask, -1e9)
        # (bs, n_head, n_q_seq, n_k_seq)
        attn_prob = nn.Softmax(dim=-1)(scores)
        attn_prob = self.dropout(attn_prob)
        # (bs, n_head, n_q_seq, d_v)
        context = torch.matmul(attn_prob, V)
        # (bs, n_head, n_q_seq, d_v), (bs, n_head, n_q_seq, n_v_seq)
        return context, attn_prob
    
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.W_Q = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.W_K = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.W_V = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.scaled_dot_attn = ScaledDotProductAttention(self.config)
        self.linear = nn.Linear(self.config.n_head * self.config.d_head, self.config.d_hidn)
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, Q, K, V, attn_mask):
        batch_size = Q.size(0)
        # (bs, n_head, n_q_seq, d_head)
        q_s = self.W_Q(Q).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)
        # (bs, n_head, n_k_seq, d_head)
        k_s = self.W_K(K).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)
        # (bs, n_head, n_v_seq, d_head)
        v_s = self.W_V(V).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)

        # (bs, n_head, n_q_seq, n_k_seq)
        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.config.n_head, 1, 1)

        # (bs, n_head, n_q_seq, d_head), (bs, n_head, n_q_seq, n_k_seq)
        context, attn_prob = self.scaled_dot_attn(q_s, k_s, v_s, attn_mask)
        # (bs, n_head, n_q_seq, h_head * d_head)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.config.n_head * self.config.d_head)
        # (bs, n_head, n_q_seq, e_embd)
        output = self.linear(context)
        output = self.dropout(output)
        # (bs, n_q_seq, d_hidn), (bs, n_head, n_q_seq, n_k_seq)
        return output, attn_prob
    
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.conv1 = nn.Conv1d(in_channels=self.config.d_hidn, out_channels=self.config.d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=self.config.d_ff, out_channels=self.config.d_hidn, kernel_size=1)
        self.active = F.gelu
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, inputs):
        # (bs, d_ff, n_seq)
        output = self.active(self.conv1(inputs.transpose(1, 2)))
        # (bs, n_seq, d_hidn)
        output = self.conv2(output).transpose(1, 2)
        output = self.dropout(output)
        # (bs, n_seq, d_hidn)
        return output

In [7]:
class EncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.self_attn = MultiHeadAttention(self.config)
        self.layer_norm1 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        self.pos_ffn = PoswiseFeedForwardNet(self.config)
        self.layer_norm2 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
    
    def forward(self, inputs, attn_mask):
        # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
        att_outputs, attn_prob = self.self_attn(inputs, inputs, inputs, attn_mask)
        att_outputs = self.layer_norm1(inputs + att_outputs)
        # (bs, n_enc_seq, d_hidn)
        ffn_outputs = self.pos_ffn(att_outputs)
        ffn_outputs = self.layer_norm2(ffn_outputs + att_outputs)
        # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
        return ffn_outputs, attn_prob
    
class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.enc_emb = nn.Embedding(self.config.n_enc_vocab, self.config.d_hidn)
        sinusoid_table = torch.FloatTensor(get_sinusoid_encoding_table(self.config.n_enc_seq + 1, self.config.d_hidn))
        self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze=True)

        self.layers = nn.ModuleList([EncoderLayer(self.config) for _ in range(self.config.n_layer)])
    
    def forward(self, inputs):
        positions = torch.arange(inputs.size(1), device=inputs.device, dtype=inputs.dtype).expand(inputs.size(0), inputs.size(1)).contiguous() + 1
        pos_mask = inputs.eq(self.config.i_pad)
        positions.masked_fill_(pos_mask, 0)

        # (bs, n_enc_seq, d_hidn)
        outputs = self.enc_emb(inputs) + self.pos_emb(positions)

        # (bs, n_enc_seq, n_enc_seq)
        attn_mask = get_attn_pad_mask(inputs, inputs, self.config.i_pad)

        attn_probs = []
        for layer in self.layers:
            # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
            outputs, attn_prob = layer(outputs, attn_mask)
            attn_probs.append(attn_prob)
        # (bs, n_enc_seq, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)]
        return outputs, attn_probs
    
class DecoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.self_attn = MultiHeadAttention(self.config)
        self.layer_norm1 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        self.dec_enc_attn = MultiHeadAttention(self.config)
        self.layer_norm2 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        self.pos_ffn = PoswiseFeedForwardNet(self.config)
        self.layer_norm3 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
    
    def forward(self, dec_inputs, enc_outputs, self_attn_mask, dec_enc_attn_mask):
        # (bs, n_dec_seq, d_hidn), (bs, n_head, n_dec_seq, n_dec_seq)
        self_att_outputs, self_attn_prob = self.self_attn(dec_inputs, dec_inputs, dec_inputs, self_attn_mask)
        self_att_outputs = self.layer_norm1(dec_inputs + self_att_outputs)
        # (bs, n_dec_seq, d_hidn), (bs, n_head, n_dec_seq, n_enc_seq)
        dec_enc_att_outputs, dec_enc_attn_prob = self.dec_enc_attn(self_att_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_enc_att_outputs = self.layer_norm2(self_att_outputs + dec_enc_att_outputs)
        # (bs, n_dec_seq, d_hidn)
        ffn_outputs = self.pos_ffn(dec_enc_att_outputs)
        ffn_outputs = self.layer_norm3(dec_enc_att_outputs + ffn_outputs)
        # (bs, n_dec_seq, d_hidn), (bs, n_head, n_dec_seq, n_dec_seq), (bs, n_head, n_dec_seq, n_enc_seq)
        return ffn_outputs, self_attn_prob, dec_enc_attn_prob
    
class Decoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.dec_emb = nn.Embedding(self.config.n_dec_vocab, self.config.d_hidn)
        sinusoid_table = torch.FloatTensor(get_sinusoid_encoding_table(self.config.n_dec_seq + 1, self.config.d_hidn))
        self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze=True)

        self.layers = nn.ModuleList([DecoderLayer(self.config) for _ in range(self.config.n_layer)])
    
    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        positions = torch.arange(dec_inputs.size(1), device=dec_inputs.device, dtype=dec_inputs.dtype).expand(dec_inputs.size(0), dec_inputs.size(1)).contiguous() + 1
        pos_mask = dec_inputs.eq(self.config.i_pad)
        positions.masked_fill_(pos_mask, 0)
    
        # (bs, n_dec_seq, d_hidn)
        dec_outputs = self.dec_emb(dec_inputs) + self.pos_emb(positions)

        # (bs, n_dec_seq, n_dec_seq)
        dec_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs, self.config.i_pad)
        # (bs, n_dec_seq, n_dec_seq)
        dec_attn_decoder_mask = get_attn_decoder_mask(dec_inputs)
        # (bs, n_dec_seq, n_dec_seq)
        dec_self_attn_mask = torch.gt((dec_attn_pad_mask + dec_attn_decoder_mask), 0)
        # (bs, n_dec_seq, n_enc_seq)
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs, self.config.i_pad)

        self_attn_probs, dec_enc_attn_probs = [], []
        for layer in self.layers:
            # (bs, n_dec_seq, d_hidn), (bs, n_dec_seq, n_dec_seq), (bs, n_dec_seq, n_enc_seq)
            dec_outputs, self_attn_prob, dec_enc_attn_prob = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            self_attn_probs.append(self_attn_prob)
            dec_enc_attn_probs.append(dec_enc_attn_prob)
        # (bs, n_dec_seq, d_hidn), [(bs, n_dec_seq, n_dec_seq)], [(bs, n_dec_seq, n_enc_seq)]S
        return dec_outputs, self_attn_probs, dec_enc_attn_probs
    
class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.encoder = Encoder(self.config)
        self.decoder = Decoder(self.config)
    
    def forward(self, enc_inputs, dec_inputs):
        # (bs, n_enc_seq, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)]
        enc_outputs, enc_self_attn_probs = self.encoder(enc_inputs)
        # (bs, n_dec_seq, d_hidn), [(bs, n_head, n_dec_seq, n_dec_seq)], [(bs, n_head, n_dec_seq, n_enc_seq)]
        dec_outputs, dec_self_attn_probs, dec_enc_attn_probs = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        # (bs, n_dec_seq, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)], [(bs, n_head, n_dec_seq, n_dec_seq)], [(bs, n_head, n_dec_seq, n_enc_seq)]
        return dec_outputs, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs

영화 분류 모델

In [8]:
class MovieClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = Transformer(self.config)
        self.projection = nn.Linear(self.config.d_hidn, self.config.n_output, bias=False)
    
    def forward(self, enc_inputs, dec_inputs):
        # (bs, n_dec_seq, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)], [(bs, n_head, n_dec_seq, n_dec_seq)], [(bs, n_head, n_dec_seq, n_enc_seq)]
        dec_outputs, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs = self.transformer(enc_inputs, dec_inputs)
        # (bs, d_hidn)
        dec_outputs, _ = torch.max(dec_outputs, dim=1)
        # (bs, n_output)
        logits = self.projection(dec_outputs)
        # (bs, n_output), [(bs, n_head, n_enc_seq, n_enc_seq)], [(bs, n_head, n_dec_seq, n_dec_seq)], [(bs, n_head, n_dec_seq, n_enc_seq)]
        return logits, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs

데이터 셋

In [9]:
class MovieDataSet(torch.utils.data.Dataset):
    def __init__(self, vocab, infile):
        self.vocab = vocab
        self.labels = []
        self.sentences = []

        df = pd.read_csv(infile)

        rating_mapping = {1: 0, 2: 1, 4: 2, 5: 3}
        self.labels = df['rating'].map(rating_mapping).tolist()
        
        docs = df['document']
        for doc in docs:
            self.sentences.append([vocab.piece_to_id(p) for p in doc])
    
    def __len__(self):
        assert len(self.labels) == len(self.sentences)
        return len(self.labels)
    
    def __getitem__(self, item):
        return (torch.tensor(self.labels[item]),
                torch.tensor(self.sentences[item]),
                torch.tensor([self.vocab.piece_to_id("[BOS]")]))

In [10]:
def movie_collate_fn(inputs):
    labels, enc_inputs, dec_inputs = list(zip(*inputs))

    enc_inputs = torch.nn.utils.rnn.pad_sequence(enc_inputs, batch_first=True, padding_value=0)
    dec_inputs = torch.nn.utils.rnn.pad_sequence(dec_inputs, batch_first=True, padding_value=0)

    batch = [
        torch.stack(labels, dim=0),
        enc_inputs,
        dec_inputs,
    ]
    return batch

In [11]:
from sklearn.model_selection import train_test_split

batch_size = 128

dataset = MovieDataSet(vocab, './nsmc_data/train.csv')

indices = list(range(len(dataset)))
train_indices, valid_indices = train_test_split(indices, test_size=0.1, random_state=42)

train_dataset = torch.utils.data.Subset(dataset, train_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=movie_collate_fn)

valid_dataset = torch.utils.data.Subset(dataset, valid_indices)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=movie_collate_fn)

test_dataset = MovieDataSet(vocab, './nsmc_data/test.csv')
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=movie_collate_fn)

In [13]:
len(train_dataset), len(valid_dataset), len(test_dataset)

(177300, 19700, 3000)

데이터 학습

In [13]:
def eval_epoch(config, model, data_loader):
    squared_errors = []
    model.eval()

    for i, value in tqdm(enumerate(data_loader), total=len(data_loader), desc="Evaluation", leave=False):
        labels, enc_inputs, dec_inputs = map(lambda v: v.to(config.device), value)

        outputs = model(enc_inputs, dec_inputs)
        logits = outputs[0]
        _, indices = logits.max(1)

        indices = torch.where(indices == 3, 5, indices)
        indices = torch.where(indices == 2, 4, indices)
        indices = torch.where(indices == 1, 2, indices)
        indices = torch.where(indices == 0, 1, indices)


        labels = torch.where(labels == 3, 5, labels)
        labels = torch.where(labels == 2, 4, labels)
        labels = torch.where(labels == 1, 2, labels)
        labels = torch.where(labels == 0, 1, labels)

        errors = indices.float() - labels.float()  # 오차 계산
        squared_errors.extend(errors.pow(2).cpu())  # 오차 제곱을 리스트에 추가

    rmse = torch.sqrt(torch.mean(torch.tensor(squared_errors)))  # 평균 제곱근 오차 계산
    return rmse.item()  # RMSE 값을 반환

In [14]:
def train_epoch(config, epoch, model, criterion, optimizer, train_loader):
    losses = []
    model.train()

    for i, value in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch}", leave=False):
        labels, enc_inputs, dec_inputs = map(lambda v: v.to(config.device), value)

        optimizer.zero_grad()
        outputs = model(enc_inputs, dec_inputs)
        logits = outputs[0]

        loss = criterion(logits, labels)
        loss_val = loss.item()
        losses.append(loss_val)

        loss.backward()
        optimizer.step()

    return np.mean(losses)

In [14]:
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config.n_output = 4 # 1,2,4,5
print(config)

learning_rate = 5e-5
n_epoch = 100

{'n_enc_vocab': 8007, 'n_dec_vocab': 8007, 'n_enc_seq': 256, 'n_dec_seq': 256, 'n_layer': 6, 'd_hidn': 256, 'i_pad': 0, 'd_ff': 1024, 'n_head': 4, 'd_head': 64, 'dropout': 0.1, 'layer_norm_epsilon': 1e-12, 'device': device(type='cpu'), 'n_output': 4}


In [16]:
model = MovieClassification(config)
model.to(config.device)

MovieClassification(
  (transformer): Transformer(
    (encoder): Encoder(
      (enc_emb): Embedding(8007, 256)
      (pos_emb): Embedding(257, 256)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): MultiHeadAttention(
            (W_Q): Linear(in_features=256, out_features=256, bias=True)
            (W_K): Linear(in_features=256, out_features=256, bias=True)
            (W_V): Linear(in_features=256, out_features=256, bias=True)
            (scaled_dot_attn): ScaledDotProductAttention(
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (linear): Linear(in_features=256, out_features=256, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (layer_norm1): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
          (pos_ffn): PoswiseFeedForwardNet(
            (conv1): Conv1d(256, 1024, kernel_size=(1,), stride=(1,))
            (conv2): Conv1d(1024, 256, kernel_size=(1,), stride=(1,))
      

In [17]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

best_epoch, best_loss, best_rmse = 0, 0, float('inf')
losses, rmses = [], []
cnt=0
for epoch in range(n_epoch):
    loss = train_epoch(config, epoch, model, criterion, optimizer, train_loader)
    rmse = eval_epoch(config, model, valid_loader)

    losses.append(loss)
    rmses.append(rmse)
    print(f'epoch : {epoch}, train loss : {loss}, valid rmse : {rmse}')

    if best_rmse > rmse:
        best_epoch, best_loss, best_rmse = epoch, loss, rmse
        torch.save(model.state_dict(), 'nsmc_rating.pth')
        cnt = 0
    else:
        cnt += 1
        if cnt == 3:
            break
    
print(f">>>> epoch = {best_epoch}, loss = {best_loss:.5f}, rmse = {best_rmse:.5f}")
test_rmse = eval_epoch(config, model, test_loader)
print(f'>>>> test rmse = {test_rmse}')

epoch : 0, train loss : 0.9151266790055609, valid rmse : 1.1724416017532349


epoch : 1, train loss : 0.8400273310587691, valid rmse : 1.1327160596847534


epoch : 2, train loss : 0.8112697686467852, valid rmse : 1.0990766286849976


epoch : 3, train loss : 0.7855242374655488, valid rmse : 1.0867218971252441


epoch : 4, train loss : 0.7645507322417365, valid rmse : 1.0594511032104492


...

epoch : 13, train loss : 0.6270587456346762, valid rmse : 1.0340644121170044


epoch : 14, train loss : 0.6118456571219115, valid rmse : 1.0519258975982666


epoch : 15, train loss : 0.5984181372548251, valid rmse : 1.0819000005722046


epoch : 16, train loss : 0.5829426982530096, valid rmse : 1.0562119483947754
>>>> epoch = 13, loss = 0.62706, rmse = 1.03406


>>>> test rmse = 1.015053391456604


내 데이터로 테스트

In [15]:
testmodel = MovieClassification(config)
testmodel.cpu()
testmodel.load_state_dict(torch.load('nsmc_rating.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [17]:
testmodel.eval()
test_input = "딸랑이는거 별로 안좋아하는편인데 이거는 굿굿입니다~~~"
test_enc_input = torch.tensor([[vocab.piece_to_id(p) for p in test_input]])
test_dec_input = torch.tensor([[vocab.piece_to_id("[BOS]")]])

print(test_enc_input)
print(test_dec_input)

test_output = testmodel(test_enc_input, test_dec_input)
test_logit = test_output[0]
_, predicted = torch.max(test_logit, 1)
print(predicted)

if predicted.item()==0:
    print(1)
elif predicted.item()==1:
    print(2)
elif predicted.item()==2:
    print(4)
else:
    print(5)


tensor([[7497, 7065, 6856, 6858, 6902,    1, 7126, 6878,    1, 6915, 6946, 6873,
         6861, 6858, 7067, 6893, 6874,    1, 6856, 6902, 6858,    1,    1,    1,
         6939, 6864, 6860, 7768, 7768, 7768]])
tensor([[2]])
tensor([1])
2


In [18]:
def compare_epoch(config, model, data_loader):
    model.eval()

    mismatched_samples = []
    er0, er1, er2 = 0, 0, 0

    for i, value in tqdm(enumerate(data_loader), total=len(data_loader), desc="compare", leave=False):
        labels, enc_inputs, dec_inputs = map(lambda v: v.to('cpu'), value)

        outputs = model(enc_inputs, dec_inputs)
        logits = outputs[0]
        _, indices = logits.max(1)

        #pred
        indices = torch.where(indices == 3, 5, indices)
        indices = torch.where(indices == 2, 4, indices)
        indices = torch.where(indices == 1, 2, indices)
        indices = torch.where(indices == 0, 1, indices)

        labels = torch.where(labels == 3, 5, labels)
        labels = torch.where(labels == 2, 4, labels)
        labels = torch.where(labels == 1, 2, labels)
        labels = torch.where(labels == 0, 1, labels)

        mismatched_differences = torch.abs(indices - labels)  # 예측값과 실제값의 차이를 절댓값으로 계산
        num_difference_0 = torch.sum(mismatched_differences == 0).item()  # 차이가 0인 개수
        num_difference_1 = torch.sum(mismatched_differences == 1).item()  # 차이가 1인 개수
        num_difference_2_plus = torch.sum(mismatched_differences >= 2).item()  # 차이가 2 이상인 개수

        er0 += num_difference_0
        er1 += num_difference_1
        er2 += num_difference_2_plus

        # indices와 labels이 다른 원소별로 enc_inputs, indices, labels 저장
        mismatched_indices = torch.where(indices != labels)[0]
        for idx in mismatched_indices:
            mismatched_samples.append((enc_inputs[idx], indices[idx], labels[idx]))


    return mismatched_samples, er0, er1, er2

In [19]:
mismatch, er0, er1, er2 = compare_epoch(config, testmodel, test_loader)

In [20]:
print(f'er0 : {er0}')
print(f'er1 : {er1}')
print(f'er2 : {er2}')

er0 : 2010
er1 : 724
er2 : 266


In [20]:
cnt1 = []
cnt2 = []
cnt4 = []
cnt5 = []
for i in range(len(mismatch)):
    if mismatch[i][2].item()==4:
        cnt4.append(i)
    elif mismatch[i][2].item()==5:
        cnt5.append(i)
    elif mismatch[i][2].item()==1:
        cnt1.append(i)
    elif mismatch[i][2].item()==2:
        cnt2.append(i)

print(cnt1)
print(cnt2)
print(cnt4)
print(cnt5)
print(len(cnt1)+len(cnt2)+len(cnt4)+len(cnt5))

[4, 5, 7, 14, 19, 27, 28, 29, 30, 31, 32, 38, 40, 49, 50, 54, 56, 57, 59, 60, 66, 70, 71, 73, 75, 78, 79, 81, 82, 84, 87, 88, 90, 91, 92, 95, 108, 111, 114, 115, 119, 123, 125, 126, 129, 130, 131, 133, 134, 135, 137, 142, 144, 145, 147, 151, 155, 159, 162, 163, 169, 170, 172, 174, 177, 180, 182, 184, 188, 191, 195, 196, 197, 198, 201, 203, 212, 213, 214, 215, 216, 217, 221, 222, 229, 230, 232, 235, 236, 238, 245, 247, 255, 257, 264, 266, 267, 268, 270, 271, 272, 273, 274, 276, 283, 284, 286, 289, 290, 291, 292, 293, 294, 296, 299, 301, 304, 305, 306, 309, 310, 311, 324, 328, 329, 331, 333, 335, 337, 338, 340, 341, 343, 346, 353, 357, 360, 362, 372, 377, 379, 381, 385, 387, 390, 391, 392, 393, 396, 400, 409, 414, 415, 416, 417, 418, 423, 425, 426, 428, 433, 434, 435, 438, 439, 447, 448, 452, 456, 459, 468, 469, 471, 473, 474, 477, 480, 481, 483, 488, 489, 493, 494, 495, 497, 499, 507, 509, 514, 515, 522, 524, 532, 533, 535, 538, 540, 542, 543, 546, 548, 551, 559, 561, 562, 567, 568, 570

In [21]:
idx = 42
print(vocab.DecodeIds(mismatch[idx][0].tolist()))
print(f'pred : {mismatch[idx][1]}')
print(f'label : {mismatch[idx][2]}')

딸랑이는거 ⁇ 별로 ⁇ 안좋아하는편인데 ⁇ 이거는 ⁇  ⁇  ⁇ 입니다~~~
pred : 2
label : 5
